In [ ]:
import os
import glob 
import json
from torch.utils.data import Dataset

In [ ]:
class ContentOrientedDataset(Dataset):
    def __init__(self, root='', crop_size=256, 
        normalize=False, **kwargs):
        super().__init__()
        
        self.data_dir = root 
        img_extensions = ['.jpg', '.png']
        self.imgs = []
        for ext in img_extensions:
            self.imgs += glob.glob(os.path.join(self.data_dir, f'images/**/*{ext}'), recursive=True)
        self.crop_size = crop_size
        self.image_dims = (3, self.crop_size, self.crop_size)
        self.normalize = normalize
        json_file_path = os.path.join(self.data_dir, "face_coords.json")
        with open(json_file_path, 'r') as json_file:
            self.face_coords = json.load(json_file)

    def _augment(self, img, face_masks, structure_masks):
        pass

    def _transforms(self, img, face_mask, structure_mask):
        pass
    
    def get_face_mask(self, idx, shape): 
        pass

    def get_structure_mask(self, idx):
        pass

    def __getitem__(self, idx):
        pass


        